# Atividade 1 - Etapa 2 - Identificar os períodos de deglutição nos arquivos válidos

In [74]:
import pandas as pd

df = pd.read_csv('arquivos.csv')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,...,-12.914958,-8.228361,-10.005320,-9.044372,-9.907545,-8.394353,-9.334880,-6.777099,-8.923019,-6.725427
1,a00002.mp3,0.579076,0.561442,0.574778,0.592735,0.597341,0.586346,0.579085,0.572200,0.559282,...,-4.921903,-4.178082,-4.070145,-4.082938,-4.426908,-4.124296,-4.116983,-3.228512,-2.619876,-1.892528
2,a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,...,-12.366119,-6.802912,-8.757007,-7.462750,-9.101726,-7.317169,-9.505013,-6.167764,-7.143456,-4.136239
3,a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,...,-1.225749,4.526946,-1.537243,0.773656,-1.744216,1.602321,-3.417079,0.585987,-1.309876,2.340520
4,a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,...,-11.525387,-6.937714,-9.290621,-9.333829,-8.506713,-6.808221,-8.480545,-6.727121,-6.899271,-4.344142


## Detector de Anomalias

In [76]:
# Anomaly Novelty Detection
from sklearn.ensemble import IsolationForest
import numpy as np


X = df[df.columns[1:]].values

clf = IsolationForest(random_state=42, contamination=0.3)
y = clf.fit_predict(X)
score = clf.score_samples(X)

df['score'] = score
np.unique(y, return_counts=True)

(array([-1,  1]), array([147, 343], dtype=int64))

In [77]:
from IPython.display import Image, Audio, display
from ipywidgets import widgets

blocos = []
for i, row in df.head(50).iterrows():
    nome = row["0"].split(".")[0]
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Anormal' if row['score'] < -0.5 else 'Normal')
        display(Audio(f'dataset/MP3/{row["0"]}'))
    blocos.append(out)
widgets.HBox(blocos)

Fica claro pelos exemplos mostrados que o detector de anomalias não consegue separar arquivos de som válidos de inválidos com deglutição.

## Classificador de Audios Válidos

Com a ajuda do Roldão, consegui um arquivo que classifica alguns arquivos de deglutição válidos

In [78]:
tutor = pd.read_csv('SomIA.csv')
tutor['Arquivo'] = tutor['Arquivo'] + '.mp3'
tutor.head()

,Arquivo,Valido
0,a00001.mp3,1
1,a00003.mp3,1
2,a00004.mp3,0
3,a00005.mp3,0
4,a00006.mp3,1


In [83]:
# conectando a tabela de parâmetros da etapa 1 com os sons válidos do tutor
ds = df.set_index('0').join(tutor.set_index('Arquivo'))
# removo as linhas que não tem identificação de válido
ds.dropna(inplace=True, axis=0)
display(ds.shape)
ds.head()

(179, 39)

,1,2,3,4,5,6,7,8,9,10,...,30,31,32,33,34,35,36,37,score,Valido
0,,,,,,,,,,,,,,,,,,,,,
a00001.mp3,0.619257,0.591173,0.588934,0.576050,0.569039,0.558396,0.591615,0.588218,0.560607,0.597746,...,-10.005320,-9.044372,-9.907545,-8.394353,-9.334880,-6.777099,-8.923019,-6.725427,-0.541842,1.0
a00003.mp3,0.659991,0.666785,0.614244,0.595386,0.595084,0.606930,0.569941,0.548508,0.509913,0.497616,...,-8.757007,-7.462750,-9.101726,-7.317169,-9.505013,-6.167764,-7.143456,-4.136239,-0.547616,1.0
a00004.mp3,0.595120,0.439680,0.370019,0.396822,0.355503,0.330981,0.379730,0.433511,0.469010,0.576902,...,-1.537243,0.773656,-1.744216,1.602321,-3.417079,0.585987,-1.309876,2.340520,-0.415591,0.0
a00005.mp3,0.600767,0.576361,0.621725,0.593066,0.625016,0.594525,0.563292,0.547832,0.565713,0.566939,...,-9.290621,-9.333829,-8.506713,-6.808221,-8.480545,-6.727121,-6.899271,-4.344142,-0.568305,0.0
a00006.mp3,0.591831,0.593079,0.578003,0.580855,0.587337,0.566847,0.557174,0.582691,0.598786,0.611115,...,-8.985192,-9.830348,-9.323259,-7.662200,-8.077006,-6.811610,-7.367786,-5.231886,-0.520624,1.0


Utilizando metodologias de aprendizado supervisionado classificador com validação cruzada (CV) para avaliação das técnicas com metrica F1:

In [84]:
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate

X = ds.drop(columns=['Valido']).values
y = ds['Valido'].values

classificadores = [
    DecisionTreeClassifier(random_state=42,max_depth=5),
    ExtraTreeClassifier(random_state=42,max_depth=5),
    RandomForestClassifier(random_state=42,max_depth=5),
    ExtraTreesClassifier(random_state=42,max_depth=5),
    GradientBoostingClassifier(random_state=42,max_depth=5),
    AdaBoostClassifier(random_state=42),
    HistGradientBoostingClassifier(random_state=42,max_depth=5),
    LogisticRegression(random_state=42),
]

resultados = []
for cls in classificadores:
    res = cross_validate(cls, X, y, cv=5, scoring='f1')
    resultados.append(
        {'metodo': cls.__class__.__name__, 
         'f1': res['test_score'].mean(), 
         'std': res['test_score'].std(), 
         'tempo': res['fit_time'].mean(),
         }
    )

df_res = pd.DataFrame(resultados)
df_res.sort_values('f1', ascending=False)

c:\Repositorio Offline\ECOM09A\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Repositorio Offline\ECOM09A\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Repositorio Offline\ECOM09A\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Repositorio Offline\ECOM09A\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent

,metodo,f1,std,tempo
2,RandomForestClassifier,0.783619,0.039088,0.142009
3,ExtraTreesClassifier,0.752681,0.079055,0.083888
4,GradientBoostingClassifier,0.749777,0.081345,0.430416
6,HistGradientBoostingClassifier,0.735278,0.070742,0.083574
5,AdaBoostClassifier,0.734617,0.051281,0.099568
0,DecisionTreeClassifier,0.688263,0.072593,0.008291
1,ExtraTreeClassifier,0.678562,0.119156,0.002755
7,LogisticRegression,0.657740,0.079289,0.015786


In [86]:
clf = RandomForestClassifier(random_state=42,max_depth=5)
clf.fit(X, y)

Xr = df[df.columns[1:]].values
y_pred = clf.predict(Xr)
df['Valido'] = y_pred
#ds.reset_index(inplace=True)
np.unique(y_pred, return_counts=True)

(array([0., 1.]), array([189, 301], dtype=int64))

In [87]:
from IPython.display import Image, Audio, display
from ipywidgets import widgets

blocos = []
for i, row in df.head(50).iterrows():
    nome = row["0"].split(".")[0]
    out = widgets.Output()
    with out:
        display(Image(f'dataset/Charts/{nome}.png'))
        display('Inválido' if row['Valido'] == 0 else 'Válido')
        display(Audio(f'dataset/MP3/{row["0"]}'))
    blocos.append(out)
widgets.HBox(blocos)